Dataset used:

[Munich Germany](https://mapzen.com/data/metro-extracts/metro/munich_germany/) from OpenStreetMap.  
[Download(OSMXML 39MB)](https://s3.amazonaws.com/metro-extracts.mapzen.com/munich_germany.osm.bz2)

将下载文件解压后命名为"munich_germany.osm"。解压后文件大小为477MB，为了简便测试，使用以下代码来生成小样本：

In [1]:
import xml.etree.cElementTree as ET  # Use cElementTree or lxml if too slow

设k为10，生成10分之一的测试用数据。

In [2]:
AREA = "munich_germany"
OSM_FILE = AREA + ".osm"  # Replace this with your osm file

k = 10 # Parameter: take every k-th top level element
SAMPLE_FILE = AREA + "_k" + str(k) + ".osm"

运行代码，生成数据大小大约为50MB。

In [3]:
def get_element(osm_file, tags=('node', 'way', 'relation')):
   """Yield element if it is the right type of tag

   Reference:
   http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
   """
   context = iter(ET.iterparse(osm_file, events=('start', 'end')))
   _, root = next(context)
   for event, elem in context:
      if event == 'end' and elem.tag in tags:
         yield elem
         root.clear()

In [4]:
from collections import defaultdict
tagElements = defaultdict(int)

with open(SAMPLE_FILE, 'wb') as output:
   output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
   output.write('<osm>\n  ')

   # Write every kth top level element
   for i, element in enumerate(get_element(OSM_FILE)):
        #if element.tag in tagElements:
        tagElements[element.tag] += 1
        #else:
        #    tagElements[element.tag] = 1
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

   output.write('</osm>')

遍历所有元素：

In [5]:
print tagElements

defaultdict(<type 'int'>, {'node': 1972102, 'relation': 8922, 'way': 391007})
